In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from sklearn.model_selection import train_test_split

from zipfile import ZipFile
import os
import sys


In [4]:
sys.path.insert(0, "../lib")

In [6]:
from resnet import *

In [8]:
cnn = ResNet18(10)

In [ ]:
cnn.load_weights